In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 7ca65eb4-83da-4479-a81c-bd303ec62374, 3, Finished, Available)

In [21]:
%%sql
create table if not exists project_lakehouse.gold_shipmode
(
 Ship_mode_ID long,
 Ship_mode string,
 Create_TS timestamp,
 Modified_TS timestamp
);

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [22]:
# incrmental load
Max_Date=spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from Project_Lakehouse.gold_shipmode").first()[0]
Max_Date

StatementMeta(, 7ca65eb4-83da-4479-a81c-bd303ec62374, 26, Finished, Available)

'1900-01-01'

In [23]:

df_bronze = spark.sql("select distinct Ship_mode from project_lakehouse.bronze_sales where Modified_TS>'{}'".format(Max_Date))
df_bronze.show()

StatementMeta(, 7ca65eb4-83da-4479-a81c-bd303ec62374, 27, Finished, Available)

+--------------+
|     Ship_mode|
+--------------+
|   First Class|
|      Same Day|
|  Second Class|
|Standard Class|
+--------------+



In [24]:
Max_ID = spark.sql("select coalesce(max(ship_mode_ID),0) from project_Lakehouse.gold_shipmode ").first()[0]
Max_ID

StatementMeta(, 7ca65eb4-83da-4479-a81c-bd303ec62374, 28, Finished, Available)

0

In [32]:
df_final = df_bronze.withColumn("ship_mode_ID",monotonically_increasing_id()+Max_ID+1)
df_final.createOrReplaceTempView("ViewShipMode")

StatementMeta(, 7ca65eb4-83da-4479-a81c-bd303ec62374, 36, Finished, Available)

In [33]:
%%sql
select * from ViewShipMode

StatementMeta(, 7ca65eb4-83da-4479-a81c-bd303ec62374, 37, Finished, Available)

<Spark SQL result set with 4 rows and 2 fields>

In [34]:
%%sql

MERGE INTO project_lakehouse.gold_shipmode AS gs
USING ViewShipMode AS vs 
    ON gs.Ship_mode = vs.Ship_mode
WHEN MATCHED THEN
    UPDATE SET
        Modified_TS = CURRENT_TIMESTAMP()
WHEN NOT MATCHED THEN
    INSERT (gs.Ship_mode_ID, gs.Ship_mode, gs.Create_TS, gs.Modified_TS)
    VALUES (vs.Ship_mode_ID, vs.Ship_mode, CURRENT_TIMESTAMP(), CURRENT_TIMESTAMP());

StatementMeta(, 7ca65eb4-83da-4479-a81c-bd303ec62374, 38, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>